In [ ]:
import os
import json
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio as rio
from rasterio.plot import show
import matplotlib.pyplot as plt
import pyproj
from matplotlib import pyplot
import rioxarray as rxr
from rasterstats import zonal_stats
import shapely
import psycopg2 
from sqlalchemy import create_engine

In [ ]:
# Load database configuration from file
with open(r'D:\iMMAP\code\db_config\hsdc_local_db_config.json', 'r') as f:
    config = json.load(f)

# Create database URL with credentials
db_url = f"postgresql://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"

# Connect to the database
con = create_engine(db_url)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
os.chdir(r'D:\iMMAP\proj\ASDC\data\pop_admin_layer')

## Preping vector data

In [ ]:
# OBS. Pop was added to admin layer prior to import
# admin_path = "SELECT * FROM afg_admbnda_adm1"  # Load from postgres
admin_sql = "select * from afg_admbnda_region_pop"           # Load from postgres
sett_path = r'D:\iMMAP\proj\ASDC\data\afg_pplp.shp'
build_sql = "select * from afg_buildings_microsoft" #r'D:\iMMAP\data\Afghanistan\buildings\microsoft_building_footprint.shp'

# Load data and convert to equal area projection
# admin = gpd.read_file(admin_path).to_crs('ESRI:53034') # Load shp
admin = gpd.GeoDataFrame.from_postgis(admin_sql, con).to_crs('ESRI:53034')   # Load postgres table
sett = gpd.read_file(sett_path).to_crs('ESRI:53034')
build = gpd.GeoDataFrame.from_postgis(build_sql, con).to_crs('ESRI:53034')    #gpd.read_file(build_path).to_crs('ESRI:53034')
# lc = gpd.read_file(lc_path)

In [ ]:
# Set admin level column (will be used for joining and aggregations)
admin_level_column = 'reg_pcode'

In [ ]:
# Reproject and save land cover
#lc.set_crs('EPSG:32642').to_crs('ESRI:53034').to_file(r'input\land-cover-dis-sample-53034.shp')

In [ ]:
# Load reprojected land cover
#lc = gpd.read_file(r'input\land-cover-dis-sample-53034.shp')

## Viewing vector data

In [ ]:
# View vector
ax = admin.plot(edgecolor='red', linewidth=2)
ax = sett.plot(ax=ax, color = 'yellow') #, facecolor='None', edgecolor='blue', linewidth=2)

## Area

In [ ]:
# Calculating area and converting from square m to square km
admin['area_km2'] = admin['geom'].area.div(1000000)

In [ ]:
admin

## Settlements

In [ ]:
# Join settlements to admin areas
joined_df = gpd.sjoin(
    sett,
    admin,
    how='inner',
    op='intersects',
)

# Count number of settlement within admin polygons
sett_count = joined_df.groupby(
    [admin_level_column],
    as_index=False,
)['vuid'].count() # vuid is arbitrary, we just count the rows

# Change column name to sett_count
sett_count.rename(columns = {'vuid': 'sett_count'}, inplace = True)

# QA Check - if any admin areas have no settlements
sett_count.isnull().sum()

# View results
sett_count

In [ ]:
# Merge sett count back on to admin dataset
admin = admin.merge(
    sett_count, 
    on=admin_level_column, 
    how='left')

admin

## Buildings

In [ ]:
# Convert building polygons to centroid points

# Convert building geodataframe to geoseries
build_centroids = build.geometry

# Create a geoseries with just the centroids
build_centroids = build_centroids.centroid

# Convert geoseries to geodataframe
build_centroids= gpd.GeoDataFrame(build_centroids)

# Rename column
build_centroids.rename(columns = {0: 'geo'}, inplace = True)

# Setting geometry columns specifically
build_centroids.geometry = build_centroids['geo']

In [ ]:
# Join building to admin areas
joined_df = gpd.sjoin(
    build_centroids,
    admin,
    how='inner',
    op='intersects'
)

# Count number of buildings within admin polygons
build_count = joined_df.groupby(
    [admin_level_column],
    as_index=False,
)['geo'].count() # vuid is arbitrary, we just count the rows

# Change column name to build_count
build_count.rename(columns = {'geo': 'build_count'}, inplace = True)

# QA Check - if any admin areas have no buildings
build_count.isnull().sum()

# View results
build_count

In [ ]:
# Merge build count back on to admin dataset
admin = admin.merge(
    build_count, 
    on=admin_level_column, 
    how='left')

## Population Density

In [ ]:
# Rename pop column
admin.rename(columns={'_sum':'pop_sum'}, inplace=True)

In [ ]:
admin

In [ ]:
# Rename pop column
admin.rename(columns={'_sum': 'pop_sum'})
#df.rename(columns={"A": "a", "B": "c"})

# Calculate population density
admin['pop_density'] = admin['pop_sum'] / admin['area_km2']

# Calculate people per building
admin['people_building'] = admin['pop_sum'] / admin['build_count']

admin

## Export results

In [ ]:
# Save to shp
#"admin.to_crs('EPSG:4326').to_file(r'output/pop_adm_full.shp')

In [ ]:
# Save to PostGIS
admin.to_postgis("afg_admbnda_region_pop_stats", con)